## Analisando os dados disponibilizados

#### Importando bibliotecas

In [1]:
# importando módulos
import pandas as pd
from unidecode import unidecode

#### Criando os DF com os dados csv

In [2]:
# lê arquivos
escolas = pd.read_csv("dados_brutos/escolas.csv")
material_didatico = pd.read_csv("dados_brutos/material_didatico.csv")
subprefeituras = pd.read_csv("dados_brutos/subprefeituras.csv")

In [3]:
print(50*"*")
print("    DF ESCOLAS")
print(50*"-")
print(escolas.info())
print(50*"-")
print("\n")
display(escolas.head(5))
display(escolas.tail(5))
print(50*"*")

**************************************************
    DF ESCOLAS
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              152 non-null    int64 
 1   Escolas_Postos  152 non-null    object
 2   BAIRRO          152 non-null    object
 3   ENDEREÇO        152 non-null    object
 4   lat             152 non-null    object
 5   lon             152 non-null    object
dtypes: int64(1), object(5)
memory usage: 7.3+ KB
None
--------------------------------------------------




,id,Escolas_Postos,BAIRRO,ENDEREÇO,lat,lon
0,178,CENTRO INTEGRADO DE EDUCAÇÃO PÚBLICA HENFIL,CAJU,Rua Carlos Seidl S/Nº,"-22,880888","-43,225326"
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,Rua Ébano 187,"-22,889574","-43,236202"
2,483,EM CELESTINO SILVA,CENTRO,"R. do Lavradio, 56","-22,909293","-43,183579"
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SÃO CRISTÓVÃO,Praça Argentina 20,"-22,897629","-43,227456"
4,132,EM PEREIRA PASSOS,RIO COMPRIDO,Praça Condessa Paulo de Frontin 45,"-22,924412","-43,208579"


,id,Escolas_Postos,BAIRRO,ENDEREÇO,lat,lon
147,474,EM ENGENHEIRO GASTÃO RANGEL,GUARATIBA,Estrada Do Magarça 9.183,"-22,98046","-43,643545"
148,301,EM JONATAS SERRANO,GUARATIBA,"Estrada Do Mato Alto, S/Nº","-22,953163","-43,577409"
149,215,E.M. NARCISA AMALIA,ILHA DE GUARATIBA,Estrada Teodoreto de Camargo S/N.º,"-23,009084","-43,537582"
150,606,EM PROFESSOR CASTILHO,ILHA DE GUARATIBA,Caminho da Matriz 4406,"-22,994124","-43,593683"
151,69,ESCOLA MUNICIPAL CARDEAL ARCOVERDE,MADUREIRA,Rua Agostinho Barbalho 401,"-22,876082","-43,343369"


**************************************************


In [4]:
print(50*"*")
print("    DF material_didatico")
print(50*"-")
print(material_didatico.info())
print(50*"-")
print("\n")
display(material_didatico.head(5))
display(material_didatico.tail(5))
print(50*"*")

**************************************************
    DF material_didatico
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          151 non-null    int64 
 1   Quantidade  147 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.5+ KB
None
--------------------------------------------------




,id,Quantidade
0,178,20
1,634,121
2,483,220
3,476,190
4,132,67


,id,Quantidade
146,474,320
147,301,335
148,215,NaN
149,606,521
150,590,225


**************************************************


In [5]:
print(50*"*")
print("    DF subprefeituras")
print(50*"-")
print(subprefeituras.info())
print(50*"-")
print("\n")
display(subprefeituras.head(5))
display(subprefeituras.tail(5))
print(50*"*")

**************************************************
    DF subprefeituras
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164 entries, 0 to 163
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   nome           164 non-null    object
 1   subprefeitura  164 non-null    object
dtypes: object(2)
memory usage: 2.7+ KB
None
--------------------------------------------------




,nome,subprefeitura
0,Santo Cristo,Centro
1,Gamboa,Centro
2,Saúde,Centro
3,Caju,Centro
4,Centro,Centro


,nome,subprefeitura
159,Santa Cruz,Zona Oeste
160,Guaratiba,Zona Oeste
161,Barra de Guaratiba,Zona Oeste
162,Pedra de Guaratiba,Zona Oeste
163,Ilha de Guaratiba,Zona Oeste


**************************************************


### Realizando a limpeza e tratamento dos dados

a.1) converter tipo coluna de objeto para inteiro

    - substituir NaN e None por 0

    - substituir '' e ' ' por 0

    - substituir 'NaN' por 0
    
    - converter valor numérico do tipo objeto para int (Ex.: '356' para 356)

#### Tratamento da tabela ```material_didatico```

In [6]:
material_didatico_copy_1 = material_didatico.copy()

In [7]:
# normalizar nomes das colunas
cols = material_didatico_copy_1.columns
for col in cols:
    material_didatico_copy_1.rename(columns={col: unidecode(col.lower().strip())}, inplace=True)

In [8]:
# remove valores nulos
material_didatico_copy_2 = material_didatico_copy_1.copy()
material_didatico_copy_2['quantidade'].fillna('0', inplace=True)

In [9]:
# converter todos os valores da coluna para int
material_didatico_copy_2['quantidade'] = material_didatico_copy_2['quantidade'].apply(lambda x: int(x) if x.isdigit() else 0)

In [10]:
# # Modo 2
# import re

# def normaliza_quantidade(valor) -> int:
#     if valor:
#         normaliza_valor = re.sub(r"[^\d: ]", "", valor.strip())
#         if normaliza_valor:
#             return int(normaliza_valor)
#         else:
#             return 0
#     else:
#         return 0
    
# material_didatico['quantidade'] = material_didatico['quantidade'].apply(lambda x: normaliza_quantidade(x))

In [11]:
material_didatico_copy_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   id          151 non-null    int64
 1   quantidade  151 non-null    int64
dtypes: int64(2)
memory usage: 2.5 KB


In [12]:
# verifica se há valores nulos
material_didatico_copy_2.isna().sum()

id            0
quantidade    0
dtype: int64

In [13]:
# verifica se há qnt igual a 0
material_didatico_copy_2[material_didatico_copy_2['quantidade'] == 0]

,id,quantidade
14,413,0
31,60,0
38,429,0
67,260,0
76,91,0
85,412,0
99,492,0
117,687,0
140,297,0
148,215,0


In [14]:
# remove linhas com valores igual a 0 na coluna quantidade
material_didatico_copy_2 = material_didatico_copy_2[material_didatico_copy_2['quantidade'] != 0]
material_didatico_copy_2

,id,quantidade
0,178,20
1,634,121
2,483,220
3,476,190
4,132,67
...,...,...
145,210,570
146,474,320
147,301,335
149,606,521


In [15]:
# verifica se há valores duplicados
material_didatico_copy_2['id'].duplicated().sum()

0

#### Tratamento da tabela ```subprefeitura```

In [16]:
subprefeituras_copy_1 = subprefeituras.copy()

In [17]:
# normalizar nomes das colunas
cols = subprefeituras_copy_1.columns
for col in cols:
    subprefeituras_copy_1.rename(columns={col: unidecode(col.lower().strip())}, inplace=True)

In [18]:
# converter letras em minúsculos e remover acentos
subprefeituras_copy_2 = subprefeituras_copy_1.copy()
subprefeituras_copy_2 = subprefeituras_copy_2.applymap(lambda x: unidecode(x.upper()))

C:\Users\wpett\AppData\Local\Temp\ipykernel_9176\603377882.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  subprefeituras_copy_2 = subprefeituras_copy_2.applymap(lambda x: unidecode(x.upper()))


In [19]:
subprefeituras_copy_2

,nome,subprefeitura
0,SANTO CRISTO,CENTRO
1,GAMBOA,CENTRO
2,SAUDE,CENTRO
3,CAJU,CENTRO
4,CENTRO,CENTRO
...,...,...
159,SANTA CRUZ,ZONA OESTE
160,GUARATIBA,ZONA OESTE
161,BARRA DE GUARATIBA,ZONA OESTE
162,PEDRA DE GUARATIBA,ZONA OESTE


In [20]:
# verifica se há valores duplicados
subprefeituras_copy_2.duplicated().sum()

0

#### Tratamento da tabela ```escolas```

In [21]:
escolas_copy_1 = escolas.copy()

In [22]:
# normalizar nomes das colunas
cols = escolas_copy_1.columns
for col in cols:
    escolas_copy_1.rename(columns={col: unidecode(col.lower().strip())}, inplace=True)

In [23]:
# # normaliza coluna id (3 caracteres. Ex: '024')
# # Modo 1
# escolas_copy_2 = escolas_copy_1.copy()
# escolas_copy_2['id'] = escolas_copy_2['id'].apply(lambda x: str(x).zfill(3))

In [24]:
# # Modo 2
# escolas_copy_2_modo2 = escolas_copy_1.copy()
# escolas_copy_2_modo2['id'] = escolas_copy_2_modo2['id'].astype(str).str.zfill(3)

In [25]:
# Modo 1
# normalizar coluna lat com 5 casas decimais
escolas_copy_3 = escolas_copy_1.copy()
for col in ['lat', 'lon']:
    escolas_copy_3[col] = escolas_copy_3[col].str.replace(",", ".").astype(float).apply(lambda x: round(x, 5))

In [26]:
# # Modo 2
# # normalizar coluna lat com 5 casas decimais
# escolas_copy_3_modo2 = escolas_copy_2.copy()
# for col in ['lat', 'lon']:
#     escolas_copy_3_modo2[col] = escolas_copy_3_modo2[col].str.replace(",", ".").astype(float).round(5)

In [27]:
escolas_copy_4 = escolas_copy_3.copy()

In [28]:
# normalizar colunas (remove acentos e string em maiúsculo)
escolas_copy_4 = escolas_copy_3.copy()
escolas_copy_4[['escolas_postos', 'bairro', 'endereco']] = escolas_copy_4[['escolas_postos', 'bairro', 'endereco']].applymap(lambda x: unidecode(x.upper()))

C:\Users\wpett\AppData\Local\Temp\ipykernel_9176\2677334891.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  escolas_copy_4[['escolas_postos', 'bairro', 'endereco']] = escolas_copy_4[['escolas_postos', 'bairro', 'endereco']].applymap(lambda x: unidecode(x.upper()))


##### Substituir abreviações de endereço

In [29]:
escolas_copy_5 = escolas_copy_4.copy()

In [30]:
# verificar possíveis abreviações
abreviacoes = set()

for i, linha_escola in escolas_copy_5.iterrows():
    lista_palavra_endereco = linha_escola['endereco'].split()
    for palavra in lista_palavra_endereco:
        if len(palavra) <= 3 and not palavra.isdigit():
            abreviacoes.update([palavra])

abreviacoes

{'ANA',
 'AV',
 'AV.',
 'DA',
 'DAS',
 'DE',
 'DO',
 'DOM',
 'DOS',
 'E',
 'JR',
 'KM',
 'PAU',
 'R.',
 'RIO',
 'RUA',
 'S/N',
 'SD.',
 'UBA'}

In [31]:
# verificar possíveis abreviações para logradouros
filtro_abreviacoes = [abreviacao for abreviacao in abreviacoes if abreviacao.startswith('R') or abreviacao.startswith('AV')]
filtro_abreviacoes

['RIO', 'AV.', 'AV', 'R.', 'RUA']

In [32]:
# normalizar coluna endereco (substituir abreviações de logradouro)
escolas_copy_5 = escolas_copy_4.copy()
escolas_copy_5['endereco'] = escolas_copy_5['endereco'].str.replace('R. ', 'RUA ', regex=True).replace('AV. ', 'AVENIDA ', regex=True).replace('AV ', 'AVENIDA ', regex=True)

In [33]:
display(escolas_copy_5.head(5))

,id,escolas_postos,bairro,endereco,lat,lon
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,RUA CARLOS SEIDL S/No,-22.88089,-43.22533
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,RUA EBANO 187,-22.88957,-43.23620
2,483,EM CELESTINO SILVA,CENTRO,"RUA DO LAVRADIO, 56",-22.90929,-43.18358
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SAO CRISTOVAO,PRACA ARGENTINA 20,-22.89763,-43.22746
4,132,EM PEREIRA PASSOS,RIO COMPRIDO,PRACA CONDESSA PAULO DE FRONTIN 45,-22.92441,-43.20858


In [34]:
escolas_copy_5.columns

Index(['id', 'escolas_postos', 'bairro', 'endereco', 'lat', 'lon'], dtype='object')

In [35]:
# valida normalização de abreviações do campo endereco
for i, linha_escola_2 in escolas_copy_5.iterrows():
    if linha_escola_2['endereco'].startswith('R. ')  or \
       linha_escola_2['endereco'].startswith('AV. ') or \
       linha_escola_2['endereco'].startswith('AV '):
        print(linha_escola_2)

##### Adicionar vírgula para o número do endereço e unificar os SN

In [159]:
import re
# Função para adicionar vírgula entre endereço e número
def add_comma_to_address(address):
    # Use expressões regulares para encontrar o número no final do endereço
    match = re.search(r'(\d+(\.\d+)?)$', address)
    
    if match:
        # Se um número for encontrado, adicione uma vírgula antes dele
        address_with_comma = re.sub(r'(\d+(\.\d+)?)$', r', \1', address)
        return address_with_comma
    else:
        return address

# Função para tratar as diversas maneiras de endereços sem numero para `SN``
def replace_s_n(text):
    # Lista de trechos a serem substituídos
    to_replace = ["S/No", "S/N.o", "S/N", "S/N.deg", "S/Ndeg", "SNdeg", "SN.deg"]
    
    # Substituir os trechos na string
    for item in to_replace:
        text = text.replace(item, ",SN")
    
    return text

In [160]:
escolas_copy_6 = escolas_copy_5.copy()

# Aplicar a função à coluna 'endereco' do DataFrame
escolas_copy_6['endereco'] = escolas_copy_6['endereco'].apply(add_comma_to_address)


escolas_copy_6.loc[:,'endereco'] = escolas_copy_6['endereco'].apply(replace_s_n)
# Exibir o DataFrame resultante
print(escolas_copy_6.head())


    id                               escolas_postos         bairro  \
0  178  CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL           CAJU   
1  634                   EM ALICE DO AMARAL PEIXOTO        BENFICA   
2  483                           EM CELESTINO SILVA         CENTRO   
3  476            ESCOLA MUNICIPAL FLORIANO PEIXOTO  SAO CRISTOVAO   
4  132                            EM PEREIRA PASSOS   RIO COMPRIDO   

                               endereco       lat       lon  
0                  RUA CARLOS SEIDL ,SN -22.88089 -43.22533  
1                       RUA EBANO , 187 -22.88957 -43.23620  
2                 RUA DO LAVRADIO, , 56 -22.90929 -43.18358  
3                  PRACA ARGENTINA , 20 -22.89763 -43.22746  
4  PRACA CONDESSA PAULO DE FRONTIN , 45 -22.92441 -43.20858  


In [161]:
escolas_copy_6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              152 non-null    int64  
 1   escolas_postos  152 non-null    object 
 2   bairro          152 non-null    object 
 3   endereco        152 non-null    object 
 4   lat             152 non-null    float64
 5   lon             152 non-null    float64
dtypes: float64(2), int64(1), object(3)
memory usage: 7.3+ KB


In [162]:
escolas_copy_6['endereco'].tail()

147          ESTRADA DO MAGARCA , 9.183
148           ESTRADA DO MATO ALTO, ,SN
149    ESTRADA TEODORETO DE CAMARGO ,SN
150            CAMINHO DA MATRIZ , 4406
151        RUA AGOSTINHO BARBALHO , 401
Name: endereco, dtype: object

In [163]:
endereco_series = escolas_copy_6['endereco']

#  str.replace() para substituir vírgulas repetidas
endereco_series = endereco_series.str.replace(r',+', ',')

# Atualize a coluna 'endereco' no DataFrame original
escolas_copy_6['endereco'] = endereco_series


In [164]:
endereco_series = escolas_copy_6['endereco']

condicao = endereco_series.str.count(',') > 1


endereco_series.loc[condicao] = endereco_series.loc[condicao].str.replace(', ', ',')
escolas_copy_6['endereco'] = endereco_series

endereco_series.loc[condicao] = endereco_series.loc[condicao].str.replace(',,', ',')
escolas_copy_6['endereco'] = endereco_series


endereco_series.loc[condicao] = endereco_series.loc[condicao].str.replace(' ,', ',')
escolas_copy_6['endereco'] = endereco_series


C:\Users\wpett\AppData\Local\Temp\ipykernel_9176\2775576906.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  endereco_series.loc[condicao] = endereco_series.loc[condicao].str.replace(', ', ',')
C:\Users\wpett\AppData\Local\Temp\ipykernel_9176\2775576906.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  endereco_series.loc[condicao] = endereco_series.loc[condicao].str.replace(',,', ',')
C:\Users\wpett\AppData\Local\Temp\ipykernel_9176\2775576906.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

In [165]:
escolas_copy_6['endereco'].head(10)

0                    RUA CARLOS SEIDL ,SN
1                         RUA EBANO , 187
2                      RUA DO LAVRADIO,56
3                    PRACA ARGENTINA , 20
4    PRACA CONDESSA PAULO DE FRONTIN , 45
5                   RUA BARAO DA TORRE,90
6                 RUA EDUARDO SANTOS , 38
7           AVENIDA PRESIDENTE VARGAS ,SN
8                     RUA ANA NERUA , 192
9                    RUA HADDOCK LOBO,148
Name: endereco, dtype: object

In [166]:
escolas_copy_6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              152 non-null    int64  
 1   escolas_postos  152 non-null    object 
 2   bairro          152 non-null    object 
 3   endereco        152 non-null    object 
 4   lat             152 non-null    float64
 5   lon             152 non-null    float64
dtypes: float64(2), int64(1), object(3)
memory usage: 7.3+ KB


##### quebrando coluna endereco em logradouro e numero

In [167]:
# Use str.split() para dividir a coluna 'endereco'
escolas_copy_6[['logradouro', 'numero']] = escolas_copy_6['endereco'].str.split(',', n=1, expand=True)

escolas_copy_6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              152 non-null    int64  
 1   escolas_postos  152 non-null    object 
 2   bairro          152 non-null    object 
 3   endereco        152 non-null    object 
 4   lat             152 non-null    float64
 5   lon             152 non-null    float64
 6   logradouro      152 non-null    object 
 7   numero          148 non-null    object 
dtypes: float64(2), int64(1), object(5)
memory usage: 9.6+ KB


In [168]:
# substitui endereços sem número informado no campo numero
escolas_copy_6['numero'].fillna('Não informado', inplace=True)

In [169]:
# verifica se preencher os campos None
display(escolas_copy_6[escolas_copy_6['numero'].isna()])

print(escolas_copy_6.info())

,id,escolas_postos,bairro,endereco,lat,lon,logradouro,numero


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              152 non-null    int64  
 1   escolas_postos  152 non-null    object 
 2   bairro          152 non-null    object 
 3   endereco        152 non-null    object 
 4   lat             152 non-null    float64
 5   lon             152 non-null    float64
 6   logradouro      152 non-null    object 
 7   numero          152 non-null    object 
dtypes: float64(2), int64(1), object(5)
memory usage: 9.6+ KB
None


In [170]:
# visualiza escola com endereço errado
display(escolas_copy_6[escolas_copy_6['id'] == 260])

# conserta endereço errado do colégio (Modo 1)
idx = escolas_copy_6.index[escolas_copy_6.id == 260]
escolas_copy_6.loc[idx, 'logradouro'] = escolas_copy_6.loc[idx]['logradouro'].str.split("4040").item()[1]
escolas_copy_6.loc[idx]

,id,escolas_postos,bairro,endereco,lat,lon,logradouro,numero
68,260,EM PROFESSOR JOSUE DE CASTRO,BONSUCESSO,AVENIDA BRASIL 4040 RUA DA VITORIA,-22.87374,-43.24071,AVENIDA BRASIL 4040 RUA DA VITORIA,Não informado


,id,escolas_postos,bairro,endereco,lat,lon,logradouro,numero
68,260,EM PROFESSOR JOSUE DE CASTRO,BONSUCESSO,AVENIDA BRASIL 4040 RUA DA VITORIA,-22.87374,-43.24071,RUA DA VITORIA,Não informado


In [171]:
# # Modo 2
# escolas_copy_6 = escolas_copy_6.loc[idx].assign(logradouro='RUA DA VITORIA')
# escolas_copy_6

## Faltam os seguintes tratamentos:

- tratar as linhas com mais de uma vírgula separando o número do endereço
- separar logradouro e número do endereço em colunas distintas


Saída esperada: 

    Arquivo csv contendo os dados no padrão especificado

    As escolas ordenadas em que os materiais didáticos devem ser entregues e com as colunas contendo as colunas:
````
['id_da_escola', 'nome_da_escola', 'tipo_da_escola', 'logradouro', 'numero', 'bairro', 'subprefeitura', 'latitude', 'longitude', 'quantidade_de_material']
````

In [172]:
# listar tipos das escolas
inicio = list()
escolas_copy_7 = escolas_copy_6.copy()
escolas_copy_7['escolas_postos'].apply(lambda x: inicio.append((x.split())[0]))
set(inicio)

{'CENTRO', 'CIEP', 'COLEGIO', 'E.M', 'E.M.', 'EM', 'EM.', 'ESCOLA'}

In [173]:
# {'CENTRO', 'CIEP', 'COLEGIO', 'E.M', 'E.M.', 'EM', 'EM.', 'ESCOLA'}
def mapear_tipo_escola(nome_da_escola: str):
    if nome_da_escola.startswith(("CIEP", "CENTRO")):
        return "CIEP"
    elif nome_da_escola.startswith("COLEGIO"):
        return "COLEGIO"
    elif nome_da_escola.replace(".", "").startswith(("ESCOLA", "EM")):
        return "EM"
    else:
        return None # Caso padrão, ou seja, nenhum valor encontrado

escolas_copy_7 = escolas_copy_6.copy()
escolas_copy_7['tipo_escola'] = escolas_copy_7['escolas_postos'].apply(mapear_tipo_escola)
escolas_copy_7

,id,escolas_postos,bairro,endereco,lat,lon,logradouro,numero,tipo_escola
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,"RUA CARLOS SEIDL ,SN",-22.88089,-43.22533,RUA CARLOS SEIDL,SN,CIEP
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,"RUA EBANO , 187",-22.88957,-43.23620,RUA EBANO,187,EM
2,483,EM CELESTINO SILVA,CENTRO,"RUA DO LAVRADIO,56",-22.90929,-43.18358,RUA DO LAVRADIO,56,EM
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SAO CRISTOVAO,"PRACA ARGENTINA , 20",-22.89763,-43.22746,PRACA ARGENTINA,20,EM
4,132,EM PEREIRA PASSOS,RIO COMPRIDO,"PRACA CONDESSA PAULO DE FRONTIN , 45",-22.92441,-43.20858,PRACA CONDESSA PAULO DE FRONTIN,45,EM
...,...,...,...,...,...,...,...,...,...
147,474,EM ENGENHEIRO GASTAO RANGEL,GUARATIBA,"ESTRADA DO MAGARCA , 9.183",-22.98046,-43.64355,ESTRADA DO MAGARCA,9.183,EM
148,301,EM JONATAS SERRANO,GUARATIBA,"ESTRADA DO MATO ALTO,SN",-22.95316,-43.57741,ESTRADA DO MATO ALTO,SN,EM
149,215,E.M. NARCISA AMALIA,ILHA DE GUARATIBA,"ESTRADA TEODORETO DE CAMARGO ,SN",-23.00908,-43.53758,ESTRADA TEODORETO DE CAMARGO,SN,EM
150,606,EM PROFESSOR CASTILHO,ILHA DE GUARATIBA,"CAMINHO DA MATRIZ , 4406",-22.99412,-43.59368,CAMINHO DA MATRIZ,4406,EM


In [174]:
# valida tratamento para obter tipo da escola
print(escolas_copy_7.info())
display(escolas_copy_7['tipo_escola'].value_counts().reset_index())
display(escolas_copy_7['tipo_escola'].value_counts(normalize=True).round(2).reset_index())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              152 non-null    int64  
 1   escolas_postos  152 non-null    object 
 2   bairro          152 non-null    object 
 3   endereco        152 non-null    object 
 4   lat             152 non-null    float64
 5   lon             152 non-null    float64
 6   logradouro      152 non-null    object 
 7   numero          152 non-null    object 
 8   tipo_escola     152 non-null    object 
dtypes: float64(2), int64(1), object(6)
memory usage: 10.8+ KB
None


,tipo_escola,count
0,EM,129
1,CIEP,22
2,COLEGIO,1


,tipo_escola,proportion
0,EM,0.85
1,CIEP,0.14
2,COLEGIO,0.01


In [175]:
# verifica quantidade de dados duplicados
print(escolas_copy_7['id'].duplicated().sum()) # escola

16


In [176]:
escolas_copy_7.drop_duplicates(inplace=True)

In [177]:
# valida se dados duplicados foram descartados
display(escolas_copy_7[escolas_copy_7.duplicated()])

,id,escolas_postos,bairro,endereco,lat,lon,logradouro,numero,tipo_escola


In [178]:
display(escolas_copy_7[escolas_copy_7['id'].duplicated(keep=False)])

,id,escolas_postos,bairro,endereco,lat,lon,logradouro,numero,tipo_escola
14,413,EM COCIO BARCELOS,COPACABANA,"RUA BARAO DE IPANEMA,34",-2.297483e+01,-4.318906e+01,RUA BARAO DE IPANEMA,34,EM
15,413,EM COCIO BARCELOS,COPACABANA,"RUA BARAO DE IPANEMA,34",-2.274828e+01,-4.318906e+01,RUA BARAO DE IPANEMA,34,EM
99,206,EM HONDURAS,PRACA SECA,"PRACA BARAO DA TAQUARUA , 12",-2.289693e+01,-4.335267e+01,PRACA BARAO DA TAQUARUA,12,EM
117,206,EM HONDURAS,PRACA SECA,"PRACA BARAO DA TAQUARUA , 12",-2.289693e+07,-4.335267e+07,PRACA BARAO DA TAQUARUA,12,EM


In [179]:
escolas_copy_7.drop_duplicates(subset='id', keep='first', inplace=True)

In [180]:
# valida remoção de todos dados duplicados
escolas_copy_7['id'].duplicated().sum()

0

In [181]:
# verifica quantidade final de escolas cadastradas
escolas_copy_7.info()

<class 'pandas.core.frame.DataFrame'>
Index: 136 entries, 0 to 150
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              136 non-null    int64  
 1   escolas_postos  136 non-null    object 
 2   bairro          136 non-null    object 
 3   endereco        136 non-null    object 
 4   lat             136 non-null    float64
 5   lon             136 non-null    float64
 6   logradouro      136 non-null    object 
 7   numero          136 non-null    object 
 8   tipo_escola     136 non-null    object 
dtypes: float64(2), int64(1), object(6)
memory usage: 10.6+ KB


In [182]:
# aplica merge entre as tabelas
escolas_copy_8 = escolas_copy_7.copy()
escolas_copy_9 = escolas_copy_8.merge(material_didatico_copy_2, how='inner', on='id')
escolas_copy_9

,id,escolas_postos,bairro,endereco,lat,lon,logradouro,numero,tipo_escola,quantidade
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,"RUA CARLOS SEIDL ,SN",-22.88089,-43.22533,RUA CARLOS SEIDL,SN,CIEP,20
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,"RUA EBANO , 187",-22.88957,-43.23620,RUA EBANO,187,EM,121
2,483,EM CELESTINO SILVA,CENTRO,"RUA DO LAVRADIO,56",-22.90929,-43.18358,RUA DO LAVRADIO,56,EM,220
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SAO CRISTOVAO,"PRACA ARGENTINA , 20",-22.89763,-43.22746,PRACA ARGENTINA,20,EM,190
4,132,EM PEREIRA PASSOS,RIO COMPRIDO,"PRACA CONDESSA PAULO DE FRONTIN , 45",-22.92441,-43.20858,PRACA CONDESSA PAULO DE FRONTIN,45,EM,67
...,...,...,...,...,...,...,...,...,...,...
121,289,EM BERTHA LUTZ,GUARATIBA,"ESTRADA DO PIAI , 2075",-22.97906,-43.67058,ESTRADA DO PIAI,2075,EM,329
122,210,EM DEBORA MENDES DE MORAES,PEDRA DE GUARATIBA,"RUA BELCHIOR DA FONSECA , 643",-23.00142,-43.63876,RUA BELCHIOR DA FONSECA,643,EM,570
123,474,EM ENGENHEIRO GASTAO RANGEL,GUARATIBA,"ESTRADA DO MAGARCA , 9.183",-22.98046,-43.64355,ESTRADA DO MAGARCA,9.183,EM,320
124,301,EM JONATAS SERRANO,GUARATIBA,"ESTRADA DO MATO ALTO,SN",-22.95316,-43.57741,ESTRADA DO MATO ALTO,SN,EM,335


In [183]:
# ordena id do df escola
list_id_escola = escolas_copy_7['id'].sort_values(ascending=True).tolist()
list_id_escola[:10]

[11, 16, 17, 20, 23, 25, 31, 35, 41, 52]

In [184]:
# ordena id do df material didatico
list_id_material = material_didatico_copy_2['id'].sort_values(ascending=True).tolist()
list_id_material[:10]

[11, 16, 17, 20, 23, 25, 31, 35, 41, 52]

In [185]:
print("Quantidade de ids (escola):", len(list_id_escola))
print("Quantidade de ids (material didático):", len(list_id_material))

Quantidade de ids (escola): 136
Quantidade de ids (material didático): 141


In [186]:
# verifica ids de material didatico não estão em escola
ids_compativeis = []
for idx in list_id_material:
    if idx not in list_id_escola:
        ids_compativeis.append(idx)

ids_compativeis

[182, 236, 271, 285, 330, 367, 410, 441, 512, 526, 590, 633, 647, 683, 698]

In [187]:
# verifica ids de escola não estão em material didático
ids_compativeis = []
for idx in list_id_escola:
    if idx not in list_id_material:
        ids_compativeis.append(idx)
    
ids_compativeis

[60, 91, 215, 260, 297, 412, 413, 429, 492, 687]

In [188]:
subprefeituras_copy_2

,nome,subprefeitura
0,SANTO CRISTO,CENTRO
1,GAMBOA,CENTRO
2,SAUDE,CENTRO
3,CAJU,CENTRO
4,CENTRO,CENTRO
...,...,...
159,SANTA CRUZ,ZONA OESTE
160,GUARATIBA,ZONA OESTE
161,BARRA DE GUARATIBA,ZONA OESTE
162,PEDRA DE GUARATIBA,ZONA OESTE


In [189]:
escolas_copy_9

,id,escolas_postos,bairro,endereco,lat,lon,logradouro,numero,tipo_escola,quantidade
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,"RUA CARLOS SEIDL ,SN",-22.88089,-43.22533,RUA CARLOS SEIDL,SN,CIEP,20
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,"RUA EBANO , 187",-22.88957,-43.23620,RUA EBANO,187,EM,121
2,483,EM CELESTINO SILVA,CENTRO,"RUA DO LAVRADIO,56",-22.90929,-43.18358,RUA DO LAVRADIO,56,EM,220
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SAO CRISTOVAO,"PRACA ARGENTINA , 20",-22.89763,-43.22746,PRACA ARGENTINA,20,EM,190
4,132,EM PEREIRA PASSOS,RIO COMPRIDO,"PRACA CONDESSA PAULO DE FRONTIN , 45",-22.92441,-43.20858,PRACA CONDESSA PAULO DE FRONTIN,45,EM,67
...,...,...,...,...,...,...,...,...,...,...
121,289,EM BERTHA LUTZ,GUARATIBA,"ESTRADA DO PIAI , 2075",-22.97906,-43.67058,ESTRADA DO PIAI,2075,EM,329
122,210,EM DEBORA MENDES DE MORAES,PEDRA DE GUARATIBA,"RUA BELCHIOR DA FONSECA , 643",-23.00142,-43.63876,RUA BELCHIOR DA FONSECA,643,EM,570
123,474,EM ENGENHEIRO GASTAO RANGEL,GUARATIBA,"ESTRADA DO MAGARCA , 9.183",-22.98046,-43.64355,ESTRADA DO MAGARCA,9.183,EM,320
124,301,EM JONATAS SERRANO,GUARATIBA,"ESTRADA DO MATO ALTO,SN",-22.95316,-43.57741,ESTRADA DO MATO ALTO,SN,EM,335


In [190]:
escolas_copy_9 = escolas_copy_9.merge(subprefeituras_copy_2, how='inner', left_on='bairro', right_on='nome')
escolas_copy_9

,id,escolas_postos,bairro,endereco,lat,lon,logradouro,numero,tipo_escola,quantidade,nome,subprefeitura
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,"RUA CARLOS SEIDL ,SN",-22.88089,-43.22533,RUA CARLOS SEIDL,SN,CIEP,20,CAJU,CENTRO
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,"RUA EBANO , 187",-22.88957,-43.23620,RUA EBANO,187,EM,121,BENFICA,CENTRO
2,600,EM URUGUAI,BENFICA,"RUA ANA NERUA , 192",-22.89849,-43.23776,RUA ANA NERUA,192,EM,591,BENFICA,CENTRO
3,483,EM CELESTINO SILVA,CENTRO,"RUA DO LAVRADIO,56",-22.90929,-43.18358,RUA DO LAVRADIO,56,EM,220,CENTRO,CENTRO
4,490,E.M TIA CIATA,CENTRO,"AVENIDA PRESIDENTE VARGAS ,SN",-22.90712,-43.19507,AVENIDA PRESIDENTE VARGAS,SN,EM,578,CENTRO,CENTRO
...,...,...,...,...,...,...,...,...,...,...,...,...
115,210,EM DEBORA MENDES DE MORAES,PEDRA DE GUARATIBA,"RUA BELCHIOR DA FONSECA , 643",-23.00142,-43.63876,RUA BELCHIOR DA FONSECA,643,EM,570,PEDRA DE GUARATIBA,ZONA OESTE
116,289,EM BERTHA LUTZ,GUARATIBA,"ESTRADA DO PIAI , 2075",-22.97906,-43.67058,ESTRADA DO PIAI,2075,EM,329,GUARATIBA,ZONA OESTE
117,474,EM ENGENHEIRO GASTAO RANGEL,GUARATIBA,"ESTRADA DO MAGARCA , 9.183",-22.98046,-43.64355,ESTRADA DO MAGARCA,9.183,EM,320,GUARATIBA,ZONA OESTE
118,301,EM JONATAS SERRANO,GUARATIBA,"ESTRADA DO MATO ALTO,SN",-22.95316,-43.57741,ESTRADA DO MATO ALTO,SN,EM,335,GUARATIBA,ZONA OESTE


In [191]:
# verifica se há valores nulos
escolas_copy_9.isna().sum()

id                0
escolas_postos    0
bairro            0
endereco          0
lat               0
lon               0
logradouro        0
numero            0
tipo_escola       0
quantidade        0
nome              0
subprefeitura     0
dtype: int64

In [192]:
# normaliza coluna id (3 caracteres. Ex: '024')
# Modo 1
escolas_copy_9['id'] = escolas_copy_9['id'].apply(lambda x: str(x).zfill(3))

# # Modo 2
# escolas_copy_9_modo2['id'] = escolas_copy_9_modo2['id'].astype(str).str.zfill(3)

In [193]:
escolas_copy_9.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              120 non-null    object 
 1   escolas_postos  120 non-null    object 
 2   bairro          120 non-null    object 
 3   endereco        120 non-null    object 
 4   lat             120 non-null    float64
 5   lon             120 non-null    float64
 6   logradouro      120 non-null    object 
 7   numero          120 non-null    object 
 8   tipo_escola     120 non-null    object 
 9   quantidade      120 non-null    int64  
 10  nome            120 non-null    object 
 11  subprefeitura   120 non-null    object 
dtypes: float64(2), int64(1), object(9)
memory usage: 11.4+ KB


In [194]:
colunas=['id_da_escola', 'nome_da_escola', 'tipo_da_escola', 'logradouro', 'numero', 'bairro', 'subprefeitura', 'latitude', 'longitude', 'quantidade_de_material']
df_final=pd.DataFrame(columns=colunas)

In [195]:
df_final['id_da_escola']=escolas_copy_9['id']
df_final['nome_da_escola']=escolas_copy_9['escolas_postos']
df_final['tipo_da_escola']=escolas_copy_9['tipo_escola']
df_final['logradouro']=escolas_copy_9['logradouro']
df_final['numero']=escolas_copy_9['numero']
df_final['bairro']=escolas_copy_9['bairro']
df_final['subprefeitura']=escolas_copy_9['subprefeitura']
df_final['latitude']=escolas_copy_9['lat']
df_final['longitude']=escolas_copy_9['lon']
df_final['quantidade_de_material']=escolas_copy_9['quantidade']

In [196]:
df_final

,id_da_escola,nome_da_escola,tipo_da_escola,logradouro,numero,bairro,subprefeitura,latitude,longitude,quantidade_de_material
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CIEP,RUA CARLOS SEIDL,SN,CAJU,CENTRO,-22.88089,-43.22533,20
1,634,EM ALICE DO AMARAL PEIXOTO,EM,RUA EBANO,187,BENFICA,CENTRO,-22.88957,-43.23620,121
2,600,EM URUGUAI,EM,RUA ANA NERUA,192,BENFICA,CENTRO,-22.89849,-43.23776,591
3,483,EM CELESTINO SILVA,EM,RUA DO LAVRADIO,56,CENTRO,CENTRO,-22.90929,-43.18358,220
4,490,E.M TIA CIATA,EM,AVENIDA PRESIDENTE VARGAS,SN,CENTRO,CENTRO,-22.90712,-43.19507,578
...,...,...,...,...,...,...,...,...,...,...
115,210,EM DEBORA MENDES DE MORAES,EM,RUA BELCHIOR DA FONSECA,643,PEDRA DE GUARATIBA,ZONA OESTE,-23.00142,-43.63876,570
116,289,EM BERTHA LUTZ,EM,ESTRADA DO PIAI,2075,GUARATIBA,ZONA OESTE,-22.97906,-43.67058,329
117,474,EM ENGENHEIRO GASTAO RANGEL,EM,ESTRADA DO MAGARCA,9.183,GUARATIBA,ZONA OESTE,-22.98046,-43.64355,320
118,301,EM JONATAS SERRANO,EM,ESTRADA DO MATO ALTO,SN,GUARATIBA,ZONA OESTE,-22.95316,-43.57741,335


In [197]:
# escreve dataframe
df_final.to_csv("dados_tratados/dados_completos_escolas.csv", index=False)

um arquivo csv no as linhas já estarão ordenas de acordo com a rota a ser seguida. Além disso, os dados devem estar no padrão especificado abaixo e contendo as seguintes colunas: id da escola, nome da escola, tipo da escola (EM, CIEP ou colégio), logradouro da entrega, número, bairro, subprefeitura, latitude, longitude e quantidade de material didático que deve ser entregue. O logradouro da escola deve estar em uma coluna diferente do número;
````
CIEP - CENTRO INTEGRADO DE EDUCACAO PUBLICA
EM - ESCOLA MUNICIPAL
COLÉGIO - COLÉGIO
`````



um arquivo csv com a quantidade total de material escolar por subprefeitura para que sejam contabilizados os custos por subprefeitura

### um arquivo csv com a quantidade total de material escolar por subprefeitura para que sejam contabilizados os custos por subprefeitura

In [198]:
qnt_material_subpref = df_final[['subprefeitura', 'quantidade_de_material']].groupby(['subprefeitura']).sum().reset_index()
qnt_material_subpref

,subprefeitura,quantidade_de_material
0,BARRA DA TIJUCA,215
1,CENTRO,2352
2,GRANDE BANGU,3979
3,GRANDE TIJUCA,2125
4,ILHAS,741
5,JACAREPAGUA,2427
6,ZONA NORTE,13088
7,ZONA OESTE,8453
8,ZONA SUL,4256


In [199]:
qnt_material_subpref.to_csv("dados_tratados/qnt_material_subprefeitura.csv", index=False)